# Generación de Data Única referente a Rol

Unificacion de la data de ingresos y terminos por Rol, además de transformación de algunos datos para normalizar nombres en columnas

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from unicodedata import normalize

from pjud.data import cleandata
from pjud.data import transformdata

In [2]:
tqdm.pandas()

path_interim = "../data/interim/pjud"
path_processed = "../data/processed/pjud"

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de TRIBUNALES OBTENIDAS DEL COT

In [3]:
# Analizo contra los datos extraidos en COT
df_tribunales = pd.read_feather(f"{path_processed}/generates_ListadoTribunales.feather")
tribunales = df_tribunales['TRIBUNAL'].unique()

In [4]:
tribunales

array(['TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL CALAMA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL COPIAPO',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL OVALLE',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL QUILLOTA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL VIÑA DEL MAR',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL VALPARAISO',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN ANTONIO',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL RANCAGUA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN FERNANDO',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTA CRUZ',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL CURICO',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL TALCA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL LINARES',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL CAUQUENES',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL CONCEPCION',


# DATA INGRESO Y TERMINO ROL PENAL

### INGRESOS X ROL

In [5]:
df_ingresos_rol = pd.read_feather(f"{path_interim}/clean_IngresosRol.feather")
df_ingresos_rol.sort_values('FECHA INGRESO')

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,MES INGRESO,AÑO INGRESO
1577082,348878,56,C.A. DE PUERTO MONTT,1091,JUZGADO DE GARANTIA DE PUERTO VARAS,1-2015,Ordinaria,2015-01-01,ene-15,2015
1315536,87181,30,C.A. DE VALPARAISO,1045,JUZGADO DE GARANTIA DE VALPARAISO,8-2015,Ordinaria,2015-01-01,ene-15,2015
1315535,87180,30,C.A. DE VALPARAISO,1045,JUZGADO DE GARANTIA DE VALPARAISO,7-2015,Ordinaria,2015-01-01,ene-15,2015
1315534,87179,30,C.A. DE VALPARAISO,1045,JUZGADO DE GARANTIA DE VALPARAISO,6-2015,Ordinaria,2015-01-01,ene-15,2015
1315533,87178,30,C.A. DE VALPARAISO,1045,JUZGADO DE GARANTIA DE VALPARAISO,5-2015,Ordinaria,2015-01-01,ene-15,2015
...,...,...,...,...,...,...,...,...,...,...
629618,23138,11,C.A. DE IQUIQUE,993,JUZGADO DE GARANTIA DE IQUIQUE,6768-2019,Ordinaria,2019-12-31,dic-19,2019
933253,326939,50,C.A. DE TEMUCO,937,JUZGADO DE GARANTIA DE TEMUCO,13161-2019,Ordinaria,2019-12-31,dic-19,2019
933255,326941,50,C.A. DE TEMUCO,937,JUZGADO DE GARANTIA DE TEMUCO,13162-2019,Ordinaria,2019-12-31,dic-19,2019
629614,23134,11,C.A. DE IQUIQUE,993,JUZGADO DE GARANTIA DE IQUIQUE,6764-2019,Ordinaria,2019-12-31,dic-19,2019


In [6]:
df_ingresos_rol['TRIBUNAL'].unique()

array(['TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA',
       'JUZGADO DE GARANTIA DE ARICA',
       'JUZGADO DE LETRAS Y GARANTIA DE POZO ALMONTE',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL DE IQUIQUE',
       'JUZGADO DE GARANTIA DE IQUIQUE',
       'JUZGADO DE LETRAS Y GARANTIA DE ALTO HOSPICIO',
       'JUZGADO DE LETRAS Y GARANTIA DE MARIA ELENA',
       'JUZGADO DE LETRAS Y GARANTIA DE TALTAL',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL DE CALAMA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ANTOFAGASTA',
       'JUZGADO DE GARANTIA DE TOCOPILLA',
       'JUZGADO DE GARANTIA DE CALAMA',
       'JUZGADO DE GARANTIA DE ANTOFAGASTA',
       'JUZGADO DE LETRAS Y GARANTIA DE MEJILLONES',
       'JUZGADO DE LETRAS Y GARANTIA DE CHAÑARAL',
       'JUZGADO DE LETRAS Y GARANTIA DE FREIRINA',
       'JUZGADO DE LETRAS Y GARANTIA DE CALDERA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL DE COPIAPO',
       'JUZGADO DE GARANTIA DE DIEGO DE ALMAGRO',
       'JUZGADO DE GARANTIA DE COPIAP

In [7]:
# Normalizo nombres

df_ingresos_rol['TRIBUNAL'] = df_ingresos_rol['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 2978550/2978550 [00:16<00:00, 184987.96it/s]


In [8]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_ingresos_rol['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

In [9]:
df_ingresos_rol['TRIBUNAL'].unique()

array(['TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA',
       'JUZGADO DE GARANTIA ARICA',
       'JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE',
       'JUZGADO DE GARANTIA IQUIQUE',
       'JUZGADO DE LETRAS Y GARANTIA ALTO HOSPICIO',
       'JUZGADO DE LETRAS Y GARANTIA MARIA ELENA',
       'JUZGADO DE LETRAS Y GARANTIA TALTAL',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL CALAMA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA',
       'JUZGADO DE GARANTIA TOCOPILLA', 'JUZGADO DE GARANTIA CALAMA',
       'JUZGADO DE GARANTIA ANTOFAGASTA',
       'JUZGADO DE LETRAS Y GARANTIA MEJILLONES',
       'JUZGADO DE LETRAS Y GARANTIA CHAÑARAL',
       'JUZGADO DE LETRAS Y GARANTIA FREIRINA',
       'JUZGADO DE LETRAS Y GARANTIA CALDERA',
       'TRIBUNAL DE JUICIO ORAL EN LO PENAL COPIAPO',
       'JUZGADO DE GARANTIA DIEGO DE ALMAGRO',
       'JUZGADO DE GARANTIA COPIAPO', 'JUZGADO DE GARANTIA VALLENAR',
       'JUZGADO DE LETRAS Y GAR

In [10]:
filtro_null = df_ingresos_rol[df_ingresos_rol['FECHA INGRESO'].isnull()]
print(f"Existen {len(filtro_null)} causas con Fechas Null")

Existen 0 causas con Fechas Null


### TERMINO X ROL

In [11]:
df_termino_rol = pd.read_feather(f"{path_interim}/clean_TerminosRol.feather")
df_termino_rol.sort_values('FECHA INGRESO')

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
400859,401124,56,C.A. DE PUERTO MONTT,662,JUZGADO DE LETRAS Y GARANTIA DE QUELLON,684-2018,Ordinaria,1900-01-01,2018-08-21,-22205,SENTENCIA,ago-18,2018,1
337694,337911,50,C.A. DE TEMUCO,214,JUZGADO DE LETRAS Y GARANTIA DE PUCON,1059-2017,Ordinaria,1900-01-01,2018-10-01,-22164,DECLARA SOBRESEIMIENTO DEFINITIVO.,oct-18,2018,1
299759,299951,46,C.A. DE CONCEPCION,1080,JUZGADO DE GARANTIA DE TALCAHUANO,2398-2017,Ordinaria,1900-01-01,2018-05-31,-22287,DECLARA SOBRESEIMIENTO DEFINITIVO.,may-18,2018,1
2766444,91242,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,1-2000,Ordinaria,2000-12-20,2015-06-02,5277,CERTIFICA CUMPLIMIENTO ART. 468.,jun-15,2015,1
2768930,93728,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,3-2000,Ordinaria,2000-12-21,2015-06-02,5276,NO PERSEVERAR EN EL PROCEDIMIENTO.,jun-15,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274585,599976,91,C.A. DE SAN MIGUEL,1229,10º JUZGADO DE GARANTIA DE SANTIAGO,4472-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
1274587,599978,91,C.A. DE SAN MIGUEL,1229,10º JUZGADO DE GARANTIA DE SANTIAGO,4473-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
1274589,599980,91,C.A. DE SAN MIGUEL,1229,10º JUZGADO DE GARANTIA DE SANTIAGO,4474-2019,Ordinaria,2019-12-31,2019-12-31,0,ACOGE REQUERIMIENTO (MONITORIO).,dic-19,2019,1
1093501,418728,56,C.A. DE PUERTO MONTT,1089,JUZGADO DE GARANTIA DE CASTRO,3116-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1


In [13]:
filtro_null = df_termino_rol[df_termino_rol['FECHA INGRESO'].isnull()]
print(f"Existen {len(filtro_null)} causas con ingresos NULL")

Existen 0 causas con ingresos NULL


In [14]:
df_termino_rol.drop(filtro_null.index, axis=0, inplace=True)

In [15]:
filtro_fecha = df_termino_rol[df_termino_rol['FECHA INGRESO'] <='2014-12-31']
print(f"Existen {len(filtro_fecha)} causas con ingresos anteriores a 01-01-2015")

Existen 410664 causas con ingresos anteriores a 01-01-2015


In [16]:
df_termino_rol.drop(filtro_fecha.index, axis=0, inplace=True)

In [17]:
# Normalizo nombres

df_termino_rol['TRIBUNAL'] = df_termino_rol['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 2944068/2944068 [00:16<00:00, 182116.31it/s]


In [18]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_termino_rol['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

In [19]:
df_termino_rol[df_termino_rol['DURACION CAUSA']<0]

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
639,639,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10153-2018,Ordinaria,2018-12-11,2018-11-30,-11,DECLARA INCOMPETENCIA.,nov-18,2018,1
640,640,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10154-2018,Ordinaria,2018-12-11,2018-11-30,-11,DECLARA INCOMPETENCIA.,nov-18,2018,1
641,641,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10155-2018,Ordinaria,2018-12-11,2018-12-03,-8,DECLARA INCOMPETENCIA.,dic-18,2018,1
642,642,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10156-2018,Ordinaria,2018-12-11,2018-12-03,-8,DECLARA INCOMPETENCIA.,dic-18,2018,1
916,916,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10413-2018,Ordinaria,2018-12-20,2018-12-05,-15,DECLARA INCOMPETENCIA.,dic-18,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352611,677837,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6341-2015,Ordinaria,2015-12-15,2015-11-20,-25,DECLARA INCOMPETENCIA.,nov-15,2015,1
3352681,677907,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6412-2015,Ordinaria,2015-12-18,2015-12-15,-3,DECLARA INCOMPETENCIA RPA,dic-15,2015,1
3352741,677967,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6497-2015,Ordinaria,2015-12-22,2015-12-17,-5,DECLARA INCOMPETENCIA RPA,dic-15,2015,1
3352801,678027,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6534-2015,Ordinaria,2015-12-22,2015-11-25,-27,DECLARA INCOMPETENCIA.,nov-15,2015,1


In [20]:
df_termino_rol = df_termino_rol.progress_apply(transformdata.fechas_cambiadas, axis=1)

100%|██████████| 2944068/2944068 [02:55<00:00, 16737.25it/s]


In [21]:
df_termino_rol[df_termino_rol['DURACION CAUSA']<0]

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS


In [22]:
df_ingresos_rol['AÑO INGRESO'].value_counts()

2019    621903
2018    606482
2017    584932
2016    583134
2015    582099
Name: AÑO INGRESO, dtype: int64

In [23]:
len(df_ingresos_rol)

2978550

## CARGA DATA A ARCHIVOS FUTHER

In [24]:
# Directorio donde se guardaran archivos feather
df_ingresos_rol.reset_index(inplace=True, drop=True)
df_termino_rol.reset_index(inplace=True, drop=True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_ingresos_rol.to_feather(f'{path_processed}/processes_IngresosRol.feather')
df_termino_rol.to_feather(f'{path_processed}/processes_TerminosRol.feather')